In [2]:
import sys
sys.path.append('..')

from datasrcs.spc import load_full_tors
from regions import load_region, SOUTH_CHASE_ALLEY
from reanalysis import ncep_r1
from plotting.mapareas import conus

%env WORKDIR=~/weatherpy-work

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime

env: WORKDIR=~/weatherpy-work


In [3]:
tors = load_full_tors(to_tz='CST')

In [4]:
init_year = 1985
fin_year = tors.yr.max()

In [5]:
latemay_tors = tors[(tors.dy > 15) & (tors.mo == 5)]

southchasealleydf, southchasealley = load_region(SOUTH_CHASE_ALLEY)
chase_tors = latemay_tors.geospatial.filter_region(southchasealley, ['slat', 'slon'])
chase_tors = chase_tors[(chase_tors.yr >= 1985) & (chase_tors.date_time.dt.hour > 12)]

In [6]:
chase_tors.tail()

,date_time,om,yr,mo,dy,tz,st,stf,stn,mag,...,len,wid,ns,sn,sg,f1,f2,f3,f4,fc
62917,2018-05-29 15:44:00,616297,2018,5,29,3,KS,20,0,0,...,4.11,100,1,1,1,69,57,0,0,0
62918,2018-05-29 16:36:00,616298,2018,5,29,3,TX,48,0,0,...,0.10,50,1,1,1,179,0,0,0,0
62919,2018-05-29 17:09:00,616299,2018,5,29,3,TX,48,0,0,...,0.10,50,1,1,1,483,0,0,0,0
62921,2018-05-29 18:25:00,616301,2018,5,29,3,OK,40,0,1,...,2.50,20,1,1,1,151,0,0,0,0
62922,2018-05-29 19:12:00,616302,2018,5,29,3,KS,20,0,0,...,0.30,30,1,1,1,157,0,0,0,0


In [7]:
tors_agg = chase_tors.groupby(['yr', 'mo', 'dy']).count()['date_time']
sigtor_agg = chase_tors[chase_tors.mag >= 2].groupby(['yr', 'mo', 'dy']).count()['date_time']

In [8]:
tordays = pd.concat([
    tors_agg[tors_agg >= 3],
    sigtor_agg
], axis=1)

In [9]:
tordays.shape

(140, 2)

In [10]:
tordays.tail()

date_time  date_time
yr   mo dy                      
2017 5  26        5.0        NaN
2018 5  18        5.0        NaN
        19        3.0        NaN
        28       22.0        NaN
        29        5.0        NaN

In [11]:
def has_tors(yr, mo, dy):
    q ='yr == @yr & mo == @mo & dy == @dy'
    tordays_queried = tordays.query(q)
    return tordays_queried.shape[0] > 0

In [12]:
datemap = {}
for yr in range(init_year, fin_year + 1):
    month = 5
    for dy in range(16, 32):
        datemap[datetime(yr, month, dy)] = has_tors(yr, month, dy)

In [13]:
dateswithtors = list(k for k in datemap if datemap[k])
dateswithouttors = list(k for k in datemap if not datemap[k])

In [14]:
len(datemap), len(dateswithtors), len(dateswithouttors)

(544, 140, 404)

In [15]:
tuple(conus)

(-127.5, -65.5, 20.5, 51)

In [16]:
def get_map_for(date_, bbox=None):
#     date_ = date_ + pd.Timedelta('1 day')
#     print(f'Fetching dataset for date: {date_} UTC')
    
    anom = ncep_r1.daily4x.hgt_anom(date_, level=500)
    ret = anom.geospatial.shiftgrid()
    if not bbox or not isinstance(bbox, tuple) or len(bbox) != 4:
        print('bbox not in the right format, not subsetting the dataset to a domain')
        return ret
    
    lon0, lon1, lat0, lat1 = bbox
    return ret.geospatial.domain(lon0, lon1, lat0, lat1)

In [17]:
# dates_ = [date_ + pd.Timedelta('1 day') for date_ in dateswithtors]
# tordate1 = get_map_for(dates_, bbox=tuple(conus))
# tordate1

In [18]:
dates_ = [date_ + pd.Timedelta('1 day') for date_ in dateswithouttors]
tordate2 = get_map_for(dates_, bbox=tuple(conus))
tordate2

/Users/jitang/anaconda/envs/storm_analysis_env/lib/python3.7/site-packages/xarray/coding/times.py:419: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  self.use_cftime)
/Users/jitang/anaconda/envs/storm_analysis_env/lib/python3.7/site-packages/numpy/core/numeric.py:538: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


<xarray.Dataset>
Dimensions:  (lat: 12, lon: 25, time: 404)
Coordinates:
    level    float32 500.0
  * lon      (lon) float32 -127.5 -125.0 -122.5 -120.0 ... -72.5 -70.0 -67.5
  * lat      (lat) float32 50.0 47.5 45.0 42.5 40.0 ... 32.5 30.0 27.5 25.0 22.5
  * time     (time) datetime64[ns] 1985-05-17 1985-05-18 ... 2018-06-01
Data variables:
    hgt      (time, lat, lon) float32 214.2002 203.7334 ... 16.1333 5.533203

In [21]:
tordate2.to_netcdf('./nontordays.nc')